In [1]:
%%writefile mapper_w.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        word,word_count = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    print >> sys.stderr, "reporter:counter:Wiki stats,Unique words,%d" % 1
    print "%s\t%s" % (word_count,word)

Overwriting mapper_w.py


In [2]:
%%writefile reducer_w.py
import sys
for line in sys.stdin:
    try:
        count,key = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    print "%s\t%d" % (key,count)

Overwriting reducer_w.py


In [3]:
%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        print "%s\t%d" % (word.lower(), 1)

Overwriting mapper.py


In [4]:
%%writefile reducer.py

import sys

current_key = None
word_sum = 0

Overwriting reducer.py


In [5]:
%%writefile -a reducer.py

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d" % (current_key, word_sum)
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print "%s\t%d" % (current_key, word_sum)

Appending to reducer.py


**Running first mapper**

In [7]:
%%bash

OUT_DIR="wordcount_result_1518514225104"
NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null 2>/dev/null
#First job to sort the files
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,reducer.py \
    -mapper "python mapper.py" \
    -combiner "python reducer.py" \
    -reducer "python reducer.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null




18/02/14 03:53:45 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/14 03:53:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/14 03:53:46 INFO mapred.FileInputFormat: Total input files to process : 1
18/02/14 03:53:46 INFO mapreduce.JobSubmitter: number of splits:2
18/02/14 03:53:46 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1518514225004_0014
18/02/14 03:53:46 INFO impl.YarnClientImpl: Submitted application application_1518514225004_0014
18/02/14 03:53:46 INFO mapreduce.Job: The url to track the job: http://71482b219d84:8088/proxy/application_1518514225004_0014/
18/02/14 03:53:46 INFO mapreduce.Job: Running job: job_1518514225004_0014
18/02/14 03:53:52 INFO mapreduce.Job: Job job_1518514225004_0014 running in uber mode : false
18/02/14 03:53:52 INFO mapreduce.Job:  map 0% reduce 0%
18/02/14 03:54:07 INFO mapreduce.Job:  map 35% reduce 0%
18/02/14 03:54:13 INFO mapreduce.Job:  map 46% reduce 0%
18/02/14 03:54:20 INFO 

**Running second mapper**

In [32]:
%%bash
OUT_DIR="wordcount_result_1518514225104"
OUT_DIR_2="wordcount_result_1518514225204"
NUM_REDUCERS=1
hdfs dfs -rm -r -skipTrash ${OUT_DIR_2} > /dev/null 2>/dev/null
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount2" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
    -D  mapred.text.key.comparator.options=-nr \
    -files mapper_w.py,reducer_w.py \
    -mapper "python mapper_w.py" \
    -reducer "python reducer_w.py" \
    -input  ${OUT_DIR}\
    -output ${OUT_DIR_2} > /dev/null



18/02/14 04:11:50 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/14 04:11:50 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/02/14 04:11:51 INFO mapred.FileInputFormat: Total input files to process : 8
18/02/14 04:11:51 INFO mapreduce.JobSubmitter: number of splits:8
18/02/14 04:11:51 INFO Configuration.deprecation: mapred.text.key.comparator.options is deprecated. Instead, use mapreduce.partition.keycomparator.options
18/02/14 04:11:51 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1518514225004_0025
18/02/14 04:11:51 INFO impl.YarnClientImpl: Submitted application application_1518514225004_0025
18/02/14 04:11:51 INFO mapreduce.Job: The url to track the job: http://71482b219d84:8088/proxy/application_1518514225004_0025/
18/02/14 04:11:51 INFO mapreduce.Job: Running job: job_1518514225004_0025
18/02/14 04:11:56 INFO mapreduce.Job: Job job_1518514225004_0025 running in uber mode : false
18/02/14 04:11:56 INFO mapreduce.Job: 

In [33]:
!hdfs dfs -cat wordcount_result_1518514225204/part* 2>/dev/null| head -7|tail -1


is	126420
